<a href="https://colab.research.google.com/github/ramana16/PnP/blob/master/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 16, 3,padding = 1),nn.BatchNorm2d(16), nn.Dropout(0.1)) #input -? OUtput? RF
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 3,padding = 1),nn.BatchNorm2d(32), nn.Dropout(0.1))
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Sequential(nn.Conv2d(32, 16, 3,padding = 1),nn.BatchNorm2d(16), nn.Dropout(0.1))
        self.conv4 = nn.Sequential(nn.Conv2d(16, 16, 3,padding = 1),nn.BatchNorm2d(16), nn.Dropout(0.1))
        self.pool2 = nn.MaxPool2d(2, 2)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.pool5 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Sequential(nn.Conv2d(16, 16, 3,padding = 1),nn.BatchNorm2d(16), nn.Dropout(0.1))
        self.conv6 = nn.Sequential(nn.Conv2d(16, 10, 3,padding = 1),nn.BatchNorm2d(10), nn.Dropout(0.1))
        #self.conv7 = nn.Sequential(nn.Conv2d(32, 10, 3,padding = 1),nn.BatchNorm2d(10), nn.Dropout(0.1))

    def forward(self, x):
        x = self.pool5(F.relu(self.conv3(self.pool3(F.relu(self.conv2(F.relu(self.conv1(x))))))))
        x = F.relu(self.pool2(self.conv6(self.pool3(F.relu(self.conv5(x))))))
        

        
        #x = F.avg_pool2d(x,(6,6))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [193]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
           Dropout-3           [-1, 16, 28, 28]               0
            Conv2d-4           [-1, 32, 28, 28]           4,640
       BatchNorm2d-5           [-1, 32, 28, 28]              64
           Dropout-6           [-1, 32, 28, 28]               0
         MaxPool2d-7           [-1, 32, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           4,624
       BatchNorm2d-9           [-1, 16, 14, 14]              32
          Dropout-10           [-1, 16, 14, 14]               0
        MaxPool2d-11             [-1, 16, 7, 7]               0
           Conv2d-12             [-1, 16, 7, 7]           2,320
      BatchNorm2d-13             [-1, 16, 7, 7]              32
          Dropout-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 16

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Streaming output truncated to the last 5000 lines.


















loss=0.1849273443222046 batch_id=3611:  96%|█████████▋| 3611/3750 [01:30<00:03, 37.04it/s]  





























loss=0.20213682949543 batch_id=3612:  96%|█████████▋| 3611/3750 [01:30<00:03, 37.04it/s]  





























loss=0.07724857330322266 batch_id=3613:  96%|█████████▋| 3611/3750 [01:30<00:03, 37.04it/s]





























loss=0.03639709949493408 batch_id=3614:  96%|█████████▋| 3611/3750 [01:30<00:03, 37.04it/s]





























loss=0.1046682596206665 batch_id=3615:  96%|█████████▋| 3611/3750 [01:30<00:03, 37.04it/s] 





























loss=0.1046682596206665 batch_id=3615:  96%|█████████▋| 3616/3750 [01:30<00:03, 40.05it/s]





























loss=0.19545389711856842 batch_id=3616:  96%|█████████▋| 3616/3750 [01:30<00:03, 40.05it/s]





























loss=0.03760981559753418 batch_id=3617:  96%|█████████▋| 3616/3750 [01:30<00:03, 40.0


Test set: Average loss: 0.1063, Accuracy: 9743/10000 (97%)



Streaming output truncated to the last 5000 lines.


















loss=0.1513836532831192 batch_id=3611:  96%|█████████▋| 3610/3750 [01:26<00:03, 43.17it/s] 





























loss=0.006633788347244263 batch_id=3612:  96%|█████████▋| 3610/3750 [01:26<00:03, 43.17it/s]





























loss=0.10218685865402222 batch_id=3613:  96%|█████████▋| 3610/3750 [01:26<00:03, 43.17it/s] 





























loss=0.10578787326812744 batch_id=3614:  96%|█████████▋| 3610/3750 [01:26<00:03, 43.17it/s]





























loss=0.10578787326812744 batch_id=3614:  96%|█████████▋| 3615/3750 [01:26<00:03, 43.87it/s]





























loss=0.006829798221588135 batch_id=3615:  96%|█████████▋| 3615/3750 [01:26<00:03, 43.87it/s]





























loss=0.036922723054885864 batch_id=3616:  96%|█████████▋| 3615/3750 [01:26<00:03, 43.87it/s]





























loss=0.12300866842269897 batch_id=3617:  96%|█████████▋| 3615/3750 [01:26<00:03,


Test set: Average loss: 0.0772, Accuracy: 9811/10000 (98%)



Streaming output truncated to the last 5000 lines.


















loss=0.025125712156295776 batch_id=3612:  96%|█████████▋| 3610/3750 [01:26<00:03, 42.46it/s]





























loss=0.4069327712059021 batch_id=3613:  96%|█████████▋| 3610/3750 [01:26<00:03, 42.46it/s]  





























loss=0.009229898452758789 batch_id=3614:  96%|█████████▋| 3610/3750 [01:26<00:03, 42.46it/s]





























loss=0.009229898452758789 batch_id=3614:  96%|█████████▋| 3615/3750 [01:26<00:03, 42.51it/s]





























loss=0.5866687297821045 batch_id=3615:  96%|█████████▋| 3615/3750 [01:26<00:03, 42.51it/s]  





























loss=0.12622418999671936 batch_id=3616:  96%|█████████▋| 3615/3750 [01:27<00:03, 42.51it/s]





























loss=0.158604234457016 batch_id=3617:  96%|█████████▋| 3615/3750 [01:27<00:03, 42.51it/s]  





























loss=0.03311377763748169 batch_id=3618:  96%|█████████▋| 3615/3750 [01:27<00:03

...

Streaming output truncated to the last 5000 lines.


















loss=0.007464706897735596 batch_id=3612:  96%|█████████▋| 3612/3750 [01:27<00:03, 40.79it/s]





























loss=0.224870964884758 batch_id=3613:  96%|█████████▋| 3612/3750 [01:28<00:03, 40.79it/s]   





























loss=0.0012060999870300293 batch_id=3614:  96%|█████████▋| 3612/3750 [01:28<00:03, 40.79it/s]





























loss=0.0011811256408691406 batch_id=3615:  96%|█████████▋| 3612/3750 [01:28<00:03, 40.79it/s]





























loss=0.0033292770385742188 batch_id=3616:  96%|█████████▋| 3612/3750 [01:28<00:03, 40.79it/s]





























loss=0.0033292770385742188 batch_id=3616:  96%|█████████▋| 3617/3750 [01:28<00:03, 41.50it/s]





























loss=0.09793664515018463 batch_id=3617:  96%|█████████▋| 3617/3750 [01:28<00:03, 41.50it/s]  





























loss=0.08226175606250763 batch_id=3618:  96%|█████████▋| 3617/3750 [01:2


Test set: Average loss: 0.0329, Accuracy: 9907/10000 (99%)



Streaming output truncated to the last 5000 lines.


















loss=0.0026344656944274902 batch_id=3612:  96%|█████████▋| 3613/3750 [01:32<00:03, 40.65it/s]





























loss=0.007968217134475708 batch_id=3613:  96%|█████████▋| 3613/3750 [01:32<00:03, 40.65it/s] 





























loss=0.003745555877685547 batch_id=3614:  96%|█████████▋| 3613/3750 [01:32<00:03, 40.65it/s]





























loss=0.03762549161911011 batch_id=3615:  96%|█████████▋| 3613/3750 [01:32<00:03, 40.65it/s] 





























loss=0.038969725370407104 batch_id=3616:  96%|█████████▋| 3613/3750 [01:32<00:03, 40.65it/s]





























loss=0.0045378804206848145 batch_id=3617:  96%|█████████▋| 3613/3750 [01:32<00:03, 40.65it/s]





























loss=0.0045378804206848145 batch_id=3617:  96%|█████████▋| 3618/3750 [01:32<00:03, 42.30it/s]





























loss=0.0015613436698913574 batch_id=3618:  96%|█████████▋| 3618/3750 [01:


Test set: Average loss: 0.0347, Accuracy: 9913/10000 (99%)



Streaming output truncated to the last 5000 lines.



















loss=0.2755436599254608 batch_id=1455:  39%|███▊      | 1451/3750 [00:35<00:51, 44.60it/s]  





























loss=0.2755436599254608 batch_id=1455:  39%|███▉      | 1456/3750 [00:35<00:52, 43.42it/s]





























loss=0.04294535517692566 batch_id=1456:  39%|███▉      | 1456/3750 [00:35<00:52, 43.42it/s]





























loss=0.005520999431610107 batch_id=1457:  39%|███▉      | 1456/3750 [00:35<00:52, 43.42it/s]





























loss=0.011381804943084717 batch_id=1458:  39%|███▉      | 1456/3750 [00:35<00:52, 43.42it/s]





























loss=0.01793709397315979 batch_id=1459:  39%|███▉      | 1456/3750 [00:35<00:52, 43.42it/s] 





























loss=0.016625791788101196 batch_id=1460:  39%|███▉      | 1456/3750 [00:35<00:52, 43.42it/s]





























loss=0.016625791788101196 batch_id=1460:  39%|███▉      | 1461/3750 [00:35<00:5